In [3]:
%pip install pandas
%pip install scikit-learn
%pip install warnings
%pip install joblib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


Note: you may need to restart the kernel to use updated packages.


Import Lybraries

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
import joblib

warnings.filterwarnings("ignore")

Import dataset

In [18]:
def import_dataset(file_path):
    return pd.read_csv(file_path)
file_path = "final-dataset.csv"

print dataset

In [19]:

dataset = import_dataset(file_path)
print(dataset.head())

                          cc_num                         merchant  \
0               3587960728692500           fraud_Simonis-Prohaska   
1  38797410180040027502291705641            fraud_Crooks and Sons   
2               2714019737356678  fraud_Ruecker, Beer and Collier   
3                            NaN       fraud_Rippin, Kub and Mann   
4                   639046421587                fraud_Gerlach Inc   

        category gender    zip      lat     long  city_pop  \
0       misc_pos      F  51006  42.3327 -95.6045      1075   
1  personal_care      M  12460  42.4069 -74.1528       277   
2   shopping_net      F  15377  39.9914 -80.4408       724   
3       misc_net      F  10162  40.7699 -73.9511   1577385   
4   shopping_net      M  15056  40.5662 -80.2099      1140   

                        job         dob                         trans_num  \
0  Teacher, early years/pre  1997-01-02  e97f0f162190b827e1bfebe86dd67685   
1     Glass blower/designer  1971-12-12  5432ea783668505d2

Feature extraction

In [20]:
def extract_features(data):
    # Convert categorical variables into numerical representations using one-hot encoding
    data = pd.get_dummies(data, columns=['merchant', 'category', 'gender', 'job'])
    # Assuming 'dob' and 'trans_num' columns are not needed for training, remove them
    data = data.drop(columns=['dob', 'trans_num'])
    return data

Extract features

In [21]:
X = extract_features(dataset.drop(columns=['is_fraud']))
y = dataset['is_fraud']
print(X.head())
print(y.head())

                          cc_num    zip      lat     long  city_pop  \
0               3587960728692500  51006  42.3327 -95.6045      1075   
1  38797410180040027502291705641  12460  42.4069 -74.1528       277   
2               2714019737356678  15377  39.9914 -80.4408       724   
3                            NaN  10162  40.7699 -73.9511   1577385   
4                   639046421587  15056  40.5662 -80.2099      1140   

   merchant_fraud_Abbott-Rogahn  merchant_fraud_Abbott-Steuber  \
0                         False                          False   
1                         False                          False   
2                         False                          False   
3                         False                          False   
4                         False                          False   

   merchant_fraud_Abernathy and Sons  merchant_fraud_Abshire PLC  \
0                              False                       False   
1                              False    

Train the Random Forest Model

In [22]:
def train_model(X_train, y_train):
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    return model

Split data into training and testing sets

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = train_model(X_train, y_train)

Model Evaluation

In [24]:

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report

Model Evaluation - Accuracy

In [25]:
accuracy = accuracy_score(y_test, model.predict(X_test))
print("Accuracy:", accuracy)
print("Classification Report:")
report = classification_report(y_test, model.predict(X_test))
print(report)


Accuracy: 0.9072337575351641
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      2688
           1       0.59      0.23      0.33       298

    accuracy                           0.91      2986
   macro avg       0.75      0.61      0.64      2986
weighted avg       0.89      0.91      0.89      2986



Save the trained model

In [26]:
joblib.dump(model, 'fraud_detection_model.pkl')
print("Model saved as fraud_detection_model.pkl")

Model saved as fraud_detection_model.pkl


Input Fields for Prediction

In [27]:
def input_field(X):
    # Prompt the user to input all the required fields
    cc_num = input("Enter credit card number: ")
    merchant = input("Enter merchant: ")
    category = input("Enter category: ")
    gender = input("Enter gender (M/F): ")
    zip_code = input("Enter zip code: ")
    lat = input("Enter latitude: ")
    long = input("Enter longitude: ")
    city_pop = input("Enter city population: ")
    job = input("Enter job: ")
    dob = input("Enter date of birth (YYYY-MM-DD): ")
    trans_num = input("Enter transaction number: ")

    # Create a DataFrame with the input data
    input_data = pd.DataFrame({
        'cc_num': [cc_num],
        'merchant': [merchant],
        'category': [category],
        'gender': [gender],
        'zip': [zip_code],
        'lat': [lat],
        'long': [long],
        'city_pop': [city_pop],
        'job': [job],
        'dob': [dob],
        'trans_num': [trans_num]
    })

    # Perform feature extraction (one-hot encoding and dropping unnecessary columns)
    input_data = extract_features(input_data)

    # Ensure all columns present during training are present in input data
    missing_cols = set(X.columns) - set(input_data.columns)
    for col in missing_cols:
        input_data[col] = 0

    # Reorder columns to match training data
    input_data = input_data[X.columns]

    return input_data


Predict and Show Output

In [28]:
def predict_show_output(model, input_data):
    prediction = model.predict(input_data)
    return "Not fraud" if prediction[0] == 0 else "Fraud"

# Get prediction for user input
input_data = input_field(X)
output = predict_show_output(model, input_data)
print("Prediction:", output)

Prediction: Secure
